In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing

In [9]:
Data = pd.read_csv('AxisBankwTech.csv', index_col = None)
Data = Data.drop(['Unnamed: 0'], axis =1)
Data.head()


,Date,Open,High,Low,Close,Adj_Close,Volume,SlowD,SlowK,WILLR,MACD,MACD_Signal,MACD_Hist,RSI,ADX,CCI,Chaikin A/D,OBV
0,2000-02-17,7.80,7.86,7.46,7.50,1.4158,282000,24.1228,19.7531,-78.5000,0.7759,0.7358,0.0401,55.0398,23.2628,-50.8876,-4.788646e+06,-1112500.0
1,2000-02-18,7.40,7.55,6.91,7.05,1.3308,448500,20.4926,20.2658,-93.5185,0.7222,0.7331,-0.0108,51.9009,21.4223,-115.7220,-5.040927e+06,-1561000.0
2,2000-02-21,7.49,7.60,7.10,7.19,1.3573,360500,19.9012,19.6848,-86.3415,0.6792,0.7223,-0.0431,52.5770,19.8626,-77.4985,-5.271647e+06,-1200500.0
3,2000-02-22,7.44,7.50,6.70,6.92,1.3063,331500,19.4578,18.4228,-90.2655,0.6337,0.7046,-0.0709,52.1247,18.1711,-111.5418,-5.420822e+06,-1532000.0
4,2000-02-23,6.78,7.14,6.60,6.74,1.2723,227500,19.3192,19.8501,-94.0678,0.5382,0.6713,-0.1331,46.2850,16.8671,-119.6643,-5.530360e+06,-1759500.0


# Creating Targets

In [4]:
Targets = pd.DataFrame()

Target1=[]
for i in range(0,len(Data['Open'])-1):
    if(Data.loc[i+1]['Open']-Data.loc[i]['Close']>=0):
        Target1.append(1)
    else:
        Target1.append(0)
Targets['Target1']=pd.Series(Target1)

Target2=[]
for i in range(0,len(Data['Open'])-1):
    if(Data.loc[i+1]['Open']-Data.loc[i]['Open']>=0):
        Target2.append(1)
    else:
        Target2.append(0)
Targets['Target2']=pd.Series(Target2)

In [4]:
Targets.head()

,Target1,Target2
0,0,0
1,1,1
2,1,0
3,0,0
4,1,1


In [5]:
#Drop the Last row
Data = Data.drop(Data.index[len(Data)-1])
Data.head()

,Date,Open,High,Low,Close,Adj_Close,Volume,SlowD,SlowK,WILLR,MACD,MACD_Signal,MACD_Hist,RSI,ADX,CCI,Chaikin A/D,OBV
0,2000-02-17,7.80,7.86,7.46,7.50,1.4158,282000,24.1228,19.7531,-78.5000,0.7759,0.7358,0.0401,55.0398,23.2628,-50.8876,-4.788646e+06,-1112500.0
1,2000-02-18,7.40,7.55,6.91,7.05,1.3308,448500,20.4926,20.2658,-93.5185,0.7222,0.7331,-0.0108,51.9009,21.4223,-115.7220,-5.040927e+06,-1561000.0
2,2000-02-21,7.49,7.60,7.10,7.19,1.3573,360500,19.9012,19.6848,-86.3415,0.6792,0.7223,-0.0431,52.5770,19.8626,-77.4985,-5.271647e+06,-1200500.0
3,2000-02-22,7.44,7.50,6.70,6.92,1.3063,331500,19.4578,18.4228,-90.2655,0.6337,0.7046,-0.0709,52.1247,18.1711,-111.5418,-5.420822e+06,-1532000.0
4,2000-02-23,6.78,7.14,6.60,6.74,1.2723,227500,19.3192,19.8501,-94.0678,0.5382,0.6713,-0.1331,46.2850,16.8671,-119.6643,-5.530360e+06,-1759500.0


In [6]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4575 entries, 0 to 4574
Data columns (total 18 columns):
Date           4575 non-null object
Open           4575 non-null float64
High           4575 non-null float64
Low            4575 non-null float64
Close          4575 non-null float64
Adj_Close      4575 non-null float64
Volume         4575 non-null int64
SlowD          4575 non-null float64
SlowK          4575 non-null float64
WILLR          4575 non-null float64
MACD           4575 non-null float64
MACD_Signal    4575 non-null float64
MACD_Hist      4575 non-null float64
RSI            4575 non-null float64
ADX            4575 non-null float64
CCI            4575 non-null float64
Chaikin A/D    4575 non-null float64
OBV            4575 non-null float64
dtypes: float64(16), int64(1), object(1)
memory usage: 679.1+ KB


#  Prediction Using TimeSeries + Technical Indicators Data

## Predicting Target 1 [Open(i+1) - Close(i)] 

### Machine Learning Models

In [7]:
X = Data
X = X.drop('Date', axis = 1)
Y = Targets['Target1']
prediction = pd.DataFrame()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = 42,shuffle='False')
X_train = preprocessing.normalize(X_train)
X_test =  preprocessing.normalize(X_test)

### Decision Tree

In [8]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
prediction['DT'] = clf.predict(X_test)
print("Accuracy "+str(accuracy_score(y_test, prediction["DT"])))
print(classification_report(y_test, prediction['DT']))
conf_mat = confusion_matrix(y_test, prediction['DT'])
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
print(conf_mat_normalized)

Accuracy 0.5450331125827814
             precision    recall  f1-score   support

          0       0.33      0.34      0.33       507
          1       0.66      0.65      0.66      1003

avg / total       0.55      0.55      0.55      1510

[[0.33530572 0.66469428]
 [0.34895314 0.65104686]]


### Logistic Regression

In [9]:
model = LogisticRegression()
model.fit(X_train,y_train)
prediction["LR"] = model.predict(X_test)
print(accuracy_score(y_test, prediction["LR"]))
print(classification_report(y_test, prediction['LR']))
conf_mat = confusion_matrix(y_test, prediction['LR'])
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
print(conf_mat_normalized)

0.6642384105960265
             precision    recall  f1-score   support

          0       0.00      0.00      0.00       507
          1       0.66      1.00      0.80      1003

avg / total       0.44      0.66      0.53      1510

[[0. 1.]
 [0. 1.]]


C:\Users\Avi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
# Support Vector Machine(SVM)
model = svm.SVC()
model.fit(X_train, y_train)  
prediction["SVM"] = model.predict(X_test)
print(accuracy_score(y_test, prediction["SVM"]))
print(classification_report(y_test, prediction['SVM']))
conf_mat = confusion_matrix(y_test, prediction['SVM'])
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
print(conf_mat_normalized)

0.6642384105960265
             precision    recall  f1-score   support

          0       0.00      0.00      0.00       507
          1       0.66      1.00      0.80      1003

avg / total       0.44      0.66      0.53      1510

[[0. 1.]
 [0. 1.]]


C:\Users\Avi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### K Nearest Neighbours(KNN)

In [13]:
model = KNeighborsClassifier(n_neighbors=3)
print(model.fit(X_train,y_train))
prediction["KNN"] = model.predict(X_test)
print("Accuracy : " + str(accuracy_score(y_test, prediction["KNN"])))
print(classification_report(y_test, prediction['KNN']))
conf_mat = confusion_matrix(y_test, prediction['KNN'])
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
print(conf_mat_normalized)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')
Accuracy : 0.5874172185430464
             precision    recall  f1-score   support

          0       0.35      0.27      0.30       507
          1       0.67      0.75      0.71      1003

avg / total       0.56      0.59      0.57      1510

[[0.26627219 0.73372781]
 [0.25024925 0.74975075]]


### Random Forest Classifier

In [14]:
rf = RandomForestClassifier(n_estimators=100)
print(rf.fit(X_train, y_train))
prediction["Random Forest Classifier"] = rf.predict(X_test)
print(accuracy_score(y_test, prediction["Random Forest Classifier"]))
print(classification_report(y_test, prediction['Random Forest Classifier']))
conf_mat = confusion_matrix(y_test, prediction['Random Forest Classifier'])
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
print(conf_mat_normalized)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.6642384105960265
             precision    recall  f1-score   support

          0       0.50      0.06      0.10       507
          1       0.67      0.97      0.79      1003

avg / total       0.61      0.66      0.56      1510

[[0.05719921 0.94280079]
 [0.02891326 0.97108674]]


### ADA Boost Classifier

In [15]:
model = AdaBoostClassifier()
print(model.fit(X_train,y_train))
prediction["ADA"] = model.predict(X_test)
print(accuracy_score(y_test, prediction["ADA"]))
print(classification_report(y_test, prediction['ADA']))
conf_mat = confusion_matrix(y_test, prediction['ADA'])
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
print(conf_mat_normalized)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)
0.6655629139072847
             precision    recall  f1-score   support

          0       0.53      0.04      0.08       507
          1       0.67      0.98      0.80      1003

avg / total       0.62      0.67      0.55      1510

[[0.04142012 0.95857988]
 [0.01894317 0.98105683]]


## Predicting Target 2 [Open(i+1) - Open(i)] 

### Machine Learning Models

In [16]:
X = Data
X = X.drop('Date', axis = 1)
Y = Targets['Target2']
prediction = pd.DataFrame()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = 42,shuffle='False')
X_train = preprocessing.normalize(X_train)
X_test =  preprocessing.normalize(X_test)

### Decision Tree

In [17]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
prediction['DT'] = clf.predict(X_test)
print("Accuracy "+str(accuracy_score(y_test, prediction["DT"])))
print(classification_report(y_test, prediction['DT']))
conf_mat = confusion_matrix(y_test, prediction['DT'])
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
print(conf_mat_normalized)

Accuracy 0.49271523178807947
             precision    recall  f1-score   support

          0       0.46      0.48      0.47       711
          1       0.52      0.50      0.51       799

avg / total       0.49      0.49      0.49      1510

[[0.47960619 0.52039381]
 [0.49561952 0.50438048]]


### Logistic Regression

In [18]:
model = LogisticRegression()
model.fit(X_train,y_train)
prediction["LR"] = model.predict(X_test)
print(accuracy_score(y_test, prediction["LR"]))
print(classification_report(y_test, prediction['LR']))
conf_mat = confusion_matrix(y_test, prediction['LR'])
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
print(conf_mat_normalized)

0.5099337748344371
             precision    recall  f1-score   support

          0       0.46      0.23      0.30       711
          1       0.53      0.76      0.62       799

avg / total       0.49      0.51      0.47      1510

[[0.22644163 0.77355837]
 [0.23779725 0.76220275]]


In [20]:
# # Support Vector Machine(SVM)

model = svm.SVC()
model.fit(X_train, y_train)  

prediction["SVM"] = model.predict(X_test)

print(accuracy_score(y_test, prediction["SVM"]))

print(classification_report(y_test, prediction['SVM']))

conf_mat = confusion_matrix(y_test, prediction['SVM'])
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
print(conf_mat_normalized)

0.5291390728476821
             precision    recall  f1-score   support

          0       0.00      0.00      0.00       711
          1       0.53      1.00      0.69       799

avg / total       0.28      0.53      0.37      1510

[[0. 1.]
 [0. 1.]]


C:\Users\Avi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### K Nearest Neighbours(KNN)

In [21]:
model = KNeighborsClassifier(n_neighbors=3)
print(model.fit(X_train,y_train))
prediction["KNN"] = model.predict(X_test)
print("Accuracy : " + str(accuracy_score(y_test, prediction["KNN"])))
print(classification_report(y_test, prediction['KNN']))
conf_mat = confusion_matrix(y_test, prediction['KNN'])
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
print(conf_mat_normalized)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')
Accuracy : 0.4735099337748344
             precision    recall  f1-score   support

          0       0.44      0.45      0.44       711
          1       0.50      0.50      0.50       799

avg / total       0.47      0.47      0.47      1510

[[0.44725738 0.55274262]
 [0.50312891 0.49687109]]


### Random Forest Classifier

In [25]:
rf = RandomForestClassifier(n_estimators=100)
print(rf.fit(X_train, y_train))
prediction["Random Forest Classifier"] = rf.predict(X_test)
print(accuracy_score(y_test, prediction["Random Forest Classifier"]))
print(classification_report(y_test, prediction['Random Forest Classifier']))
conf_mat = confusion_matrix(y_test, prediction['Random Forest Classifier'])
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
print(conf_mat_normalized)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.7165562913907285
             precision    recall  f1-score   support

          0       0.69      0.71      0.70       711
          1       0.74      0.72      0.73       799

avg / total       0.72      0.72      0.72      1510

[[0.71448664 0.28551336]
 [0.281602   0.718398  ]]


### ADA Boost Classifier

In [26]:
model = AdaBoostClassifier()
print(model.fit(X_train,y_train))
prediction["ADA"] = model.predict(X_test)
print(accuracy_score(y_test, prediction["ADA"]))
print(classification_report(y_test, prediction['ADA']))
conf_mat = confusion_matrix(y_test, prediction['ADA'])
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
print(conf_mat_normalized)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)
0.7185430463576159
             precision    recall  f1-score   support

          0       0.70      0.70      0.70       711
          1       0.73      0.74      0.73       799

avg / total       0.72      0.72      0.72      1510

[[0.697609   0.302391  ]
 [0.26282854 0.73717146]]
